![](http://nipype.readthedocs.io/en/latest/_static/nipype-banner-bg.png)

# What is Nipype?

[Nipype](http://nipype.readthedocs.io/en/latest/) (Neuroimaging in Python - Pipelines and Interfaces) is an open-source, user-friendly, community-developed software package under the umbrella of [NiPy](http://nipy.org/). Nipype allows you to pipeline your neuroimaging workflow in an intuitive way and enables you to use the software packages and algorithms you want to use, regardless their programing language. This is possible because Nipype provides an uniform interface to many existing neuroimaging processing and analysis packages like [ANTS](http://stnava.github.io/ANTs/), [SPM](http://www.fil.ion.ucl.ac.uk/spm/), [FSL](http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/), [FreeSurfer](http://surfer.nmr.mgh.harvard.edu/), [Camino](http://web4.cs.ucl.ac.uk/research/medic/camino/pmwiki/pmwiki.php), [MRtrix](http://www.brain.org.au/software/mrtrix/index.html), [MNE](https://martinos.org/mne/stable/index.html), [AFNI](https://afni.nimh.nih.gov/afni), [Slicer](https://www.slicer.org/) and many more. Nipype allows you to easily combine all those heterogeneous software packages within a single workflow. This procedure gives you the opportunity to pick the best algorithm there is for the problem at hand and therefore allows you to profit from the advantages of any software package you like.

Nipype is written in [Python](https://www.python.org/), an easy to learn and very intuitive programming language. This ensures a rather steep learning curve and makes you a Nipype expert in no time. Nipype provides an environment that encourages interactive exploration of algorithms. It allows you to make your research easily reproducible and lets you share your code with the community. And for any problem you can encounter, there's always a community standing behind you.

# Short Example

Let's assume you want to do preprocessing that uses **SPM** for *motion correction*, **FreeSurfer** for *coregistration*, **ANTS** for *normalization* and **FSL** for *smoothing*. Normally this would be a hell of a mess. Switching between multiple scripts in different programming languages with a lot of manual intervention. On top of all that, you want to do your analysis on multiple subjects, preferably as fast as possible, i.e., processing several subjects in parallel. With Nipype, this is no problem! You can do all this and much more.

To illustrate the straightforwardness of an Nipype workflow and show how simply it can be created, look at the following example. This figure shows you a simplification of the analysis *workflow* just outlined.

![](../static/images/nipype_example_graph.png)

The code to create an Nipype workflow that specifies the steps illustrated in the figure above would look something like this:

```python
# Import modules
import nipype
import nipype.interfaces.freesurfer as fs
import nipype.interfaces.ants       as ants
import nipype.interfaces.fsl        as fsl
import nipype.interfaces.spm        as spm

# Specify experiment specifc parameters
experiment_dir = '~/experiment_folder'
nameofsubjects = ['subject1','subject2','subject3']

# Where can the raw data be found?
grabber = nipype.DataGrabber()
grabber.inputs.base_directory = experiment_dir + '/data'
grabber.inputs.subject_id = nameofsubjects

# Where should the output data be stored at?
sink = nipype.DataSink()
sink.inputs.base_directory = experiment_dir + '/output_folder'

# Motion Correction (SPM)
realign = spm.Realing(register_to_mean=True)

# Coregistration (FreeSurfer)
coreg = fs.BBRegister()

# Normalization (ANTS)
normalize = ants.WarpTimeSeriesImageMultiTransform()

# Smoothing (FSL)
smooth = fsl.SUSAN(fwhm=6.0)

# Create a workflow to connect all those nodes
preprocflow = nipype.Workflow()

# Connect the nodes to each other
preprocflow.connect([(grabber   -> realign  ),
                     (realign   -> coreg    ),
                     (coreg     -> normalize),
                     (normalize -> smooth   ),
                     (smooth    -> sink     )
                     ])

# Run the workflow in parallel
preprocflow.run(mode='parallel')
```

**Important**: The code above is of course a shortened and simplified version of the real code. But it gives you a good idea of what the code would look like, and how straightforward and readable the programming of a neuroimaging pipeline with Nipype is.

# Nipype's Architecture

Nipype consists of many parts, but the most important ones are [Interfaces](basic_interfaces.ipynb), the [Workflow Engine](basic_workflow.ipynb)  and the [Execution Plugins](basic_plugins.ipynb):

* **Interface**: Wraps a program or function

* **Node/MapNode**: Wraps an `Interface` for use in a Workflow that provides caching and other goodies (e.g., pseudo-sandbox)
* **Workflow**: A *graph* or *forest of graphs* whose nodes are of type `Node`, `MapNode` or `Workflow` and whose edges represent data flow

* **Plugin**: A component that describes how a `Workflow` should be executed

![](../static/images/nipype_architecture.png)

# Reference

Gorgolewski, K., Burns, C. D., Madison, C., Clark, D., Halchenko, Y. O., Waskom, M. L., & Ghosh, S. S. (2011). Nipype: a flexible, lightweight and extensible neuroimaging data processing framework in python. *Frontiers in neuroinformatics, 5*, 13. [Download paper](http://journal.frontiersin.org/article/10.3389/fninf.2011.00013)